# Get GDELT Schema

In [107]:
import urllib.request  # lib that handles URLs
#import requests
import io
import pandas as pd
pd.options.display.max_rows=250
import numpy as np

target_url="https://raw.githubusercontent.com/linwoodc3/gdelt2HeaderRows/master/schema_csvs/GDELT_2.0_Events_Column_Labels_Header_Row_Sep2016.csv"
data = urllib.request.urlopen(target_url).read().decode('utf8')
formats = pd.read_csv(io.StringIO(data))
colnames_events = list(formats["tableId"])

target_url="https://raw.githubusercontent.com/linwoodc3/gdelt2HeaderRows/master/schema_csvs/GDELT_2.0_eventMentions_Column_Labels_Header_Row_Sep2016.tsv"
data = urllib.request.urlopen(target_url).read().decode('utf8')
formats = pd.read_csv(io.StringIO(data),sep="\t")
colnames_mentions = list(formats.loc[:15,"0"])

target_url="https://raw.githubusercontent.com/linwoodc3/gdelt2HeaderRows/master/schema_csvs/GDELT_2.0_gdeltKnowledgeGraph_Column_Labels_Header_Row_Sep2016.tsv"
data = urllib.request.urlopen(target_url).read().decode('utf8')
formats = pd.read_csv(io.StringIO(data),sep="\t")
colnames_gkg = list(formats["tableId"])

# Dataset Imports

In [3]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder \
     .appName("Test") \
     .getOrCreate()

In [4]:
df_events = spark.read.format("csv") \
    .option("header", "false") \
    .option("delimiter","\t") \
    .option("inferSchema", "true") \
    .load("s3://gdelt-open-data/v2/events/20180101*.export.csv")
df_events = df_events.toDF(*colnames_events)
df_events.cache()

DataFrame[GLOBALEVENTID: int, SQLDATE: int, MonthYear: int, Year: int, FractionDate: double, Actor1Code: string, Actor1Name: string, Actor1CountryCode: string, Actor1KnownGroupCode: string, Actor1EthnicCode: string, Actor1Religion1Code: string, Actor1Religion2Code: string, Actor1Type1Code: string, Actor1Type2Code: string, Actor1Type3Code: string, Actor2Code: string, Actor2Name: string, Actor2CountryCode: string, Actor2KnownGroupCode: string, Actor2EthnicCode: string, Actor2Religion1Code: string, Actor2Religion2Code: string, Actor2Type1Code: string, Actor2Type2Code: string, Actor2Type3Code: string, IsRootEvent: int, EventCode: int, EventBaseCode: int, EventRootCode: int, QuadClass: int, GoldsteinScale: double, NumMentions: int, NumSources: int, NumArticles: int, AvgTone: double, Actor1Geo_Type: int, Actor1Geo_FullName: string, Actor1Geo_CountryCode: string, Actor1Geo_ADM1Code: string, Actor1Geo_ADM2Code: string, Actor1Geo_Lat: double, Actor1Geo_Long: double, Actor1Geo_FeatureID: string,

In [9]:
df_mentions = spark.read.format("csv") \
    .option("header", "false") \
    .option("delimiter","\t") \
    .option("inferSchema", "true") \
    .load("s3://gdelt-open-data/v2/mentions/20180101*.mentions.csv")
df_mentions = df_mentions.toDF(*colnames_mentions)
df_mentions.cache()

DataFrame[GLOBALEVENTID: int, EventTimeDate: bigint, MentionTimeDate: bigint, MentionType: int, MentionSourceName: string, MentionIdentifier: string, SentenceID: int, Actor1CharOffset: int, Actor2CharOffset: int, ActionCharOffset: int, InRawText: int, Confidence: int, MentionDocLen: int, MentionDocTone: double, MentionDocTranslationInfo: string, Extras: string]

In [218]:
from datetime import datetime
start = datetime.now()

df_gkg = spark.read.format("csv") \
    .option("header", "false") \
    .option("delimiter","\t") \
    .option("inferSchema", "true") \
    .load(path=['s3://gdelt-open-data/v2/gkg/20171031*.gkg.csv',
                's3://gdelt-open-data/v2/gkg/20171101*.gkg.csv',
                's3://gdelt-open-data/v2/gkg/20180122*.gkg.csv',
                's3://gdelt-open-data/v2/gkg/20180123*.gkg.csv',
                's3://gdelt-open-data/v2/gkg/20180205*.gkg.csv',
                's3://gdelt-open-data/v2/gkg/20180206*.gkg.csv',
                's3://gdelt-open-data/v2/gkg/20180417*.gkg.csv',
                's3://gdelt-open-data/v2/gkg/20180418*.gkg.csv',
                's3://gdelt-open-data/v2/gkg/20180514*.gkg.csv',
                's3://gdelt-open-data/v2/gkg/20180515*.gkg.csv',
                's3://gdelt-open-data/v2/gkg/20180518*.gkg.csv',
                's3://gdelt-open-data/v2/gkg/20180519*.gkg.csv',
                's3://gdelt-open-data/v2/gkg/20180814*.gkg.csv',
                's3://gdelt-open-data/v2/gkg/20180815*.gkg.csv'
               ])
df_gkg = df_gkg.toDF(*colnames_knowledgegraph)
df_gkg.cache()

print(datetime.now()-start)

0:04:50.966263


In [219]:
start = datetime.now()

# print(df_events.count())
# print(df_mentions.count())

print(df_gkg.count())

#print(df_events.printSchema())

print(datetime.now()-start)

3590283
0:02:08.390906


# Basic Queries
* https://blog.gdeltproject.org/gdelt-2-0-our-global-world-in-realtime/
* https://www.piie.com/system/files/documents/trump-trade-war-timeline.pdf
* https://www.bbvaresearch.com/wp-content/uploads/2018/07/Exploring-the-global-trade-and-protectionism-in-real-time-using-Big-Data_.pdf
* https://www.bbvaresearch.com/wp-content/uploads/2018/09/Trade_Wars_Asia_Eng_.pdf

In [224]:
#test_gkg = df_gkg.where((df_gkg.V2Persons.rlike('Donald Trump')))
topic = "([T|t]rade|[T|t]ariff)"

test_gkg = df_gkg.where(
    (
        (df_gkg.DocumentIdentifier.rlike(topic)) 
        | (df_gkg.Quotations.rlike(topic))
    )
    &
    (
        (df_gkg.V2Persons.rlike('Donald Trump'))
    )
)

test = test_gkg #.join(df_events, df_events.SOURCEURL == test_gkg.DocumentIdentifier, how="left")

from pyspark.sql.functions import regexp_replace
test = test.withColumn("V2Themes", regexp_replace("V2Themes",',[0-9]*',""))

test.cache()

print(test_gkg.count())

10214


In [225]:
recs = pd.DataFrame(test.collect())
recs.columns = colnames_gkg #+ colnames_events 
recs.shape
#print(np.array(recs.loc[:,][["Persons","V2Themes"]].drop_duplicates()))

(10214, 27)

In [226]:
np.array(recs.tail(5)["DocumentIdentifier"])

array(['http://english.ahram.org.eg/NewsContent/2/8/299781/World/Region/Muslim-leaders-call-for-international-protection-f.aspx',
       'https://www.morningstar.com/news/dow-jones/TDJNDN_201805185914/market-snapshot-stock-market-struggles-for-direction-with-trade-talks-yields-in-the-spotlight.html',
       'https://www.bloomberg.com/news/articles/2018-05-19/merkel-says-she-ll-seek-china-as-free-trade-ally-on-beijing-trip',
       'https://www.tasnimnews.com/en/news/2018/05/19/1729686/eu-commissioner-in-iran-to-discuss-trade-ties-after-us-jcpoa-pullout',
       'https://abcnews.go.com/Business/wireStory/correction-united-states-china-zte-story-55150862'],
      dtype=object)

In [166]:
recs = pd.DataFrame(test_gkg.select("V2Themes").distinct().collect())
set(";".join(np.array(recs.head(10)).flatten()).split(";"))

{'',
 'AFFECT',
 'ALLIANCE',
 'APPOINTMENT',
 'ARMEDCONFLICT',
 'ARREST',
 'AUSTERITY',
 'BORDER',
 'CEASEFIRE',
 'CONSTITUTIONAL',
 'CORRUPTION',
 'CRISISLEX_C07_SAFETY',
 'CRISISLEX_CRISISLEXREC',
 'CRISISLEX_T01_CAUTION_ADVICE',
 'CRISISLEX_T02_INJURED',
 'CRISISLEX_T03_DEAD',
 'CRISISLEX_T08_MISSINGFOUNDTRAPPEDPEOPLE',
 'CRISISLEX_T11_UPDATESSYMPATHY',
 'DEMOCRACY',
 'ECON_STOCKMARKET',
 'EDUCATION',
 'ELECTION',
 'EPU_CATS_MIGRATION_FEAR_FEAR',
 'EPU_CATS_NATIONAL_SECURITY',
 'EPU_CATS_TAXES',
 'EPU_ECONOMY_HISTORIC',
 'EPU_POLICY_AUTHORITIES',
 'EPU_POLICY_GOVERNMENT',
 'EPU_POLICY_POLITICAL',
 'EPU_POLICY_WHITE_HOUSE',
 'FOOD_SECURITY',
 'GENERAL_GOVERNMENT',
 'GENERAL_HEALTH',
 'GENERAL_PUBLIC_OPINION',
 'GEN_HOLIDAY',
 'HARASSMENT',
 'HUMAN_RIGHTS_ABUSES_TORTURING',
 'KILL',
 'LEADER',
 'LEGISLATION',
 'LGBT',
 'MANMADE_DISASTER_IMPLIED',
 'MARITIME',
 'MARITIME_INCIDENT',
 'MEDIA_MSM',
 'MEDIA_SOCIAL',
 'MEDICAL',
 'MILITARY',
 'MIL_MERCENARIES',
 'NATURAL_DISASTER_WINTRY',
 

# Stop Spark Context

In [ ]:
sc.stop()